<a href="https://colab.research.google.com/github/ubiodee/Plutus_Demo/blob/main/Cardano_Plutus_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastapi import FastAPI
from fastapi.responses import FileResponse
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok
import nest_asyncio

# Apply fixes for running FastAPI in Google Colab
nest_asyncio.apply()

# ✅ Set your Ngrok authentication token here
NGROK_AUTH_TOKEN = ""  # Replace with your token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Define the model path (your trained model on Hugging Face)
MODEL_NAME = "ubiodee/Cardano_plutus"

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Set up FastAPI app
app = FastAPI()

# ✅ Root Route (Now Handles `/favicon.ico`)
@app.get("/")
async def root():
    return {"message": "Welcome to the Cardano Plutus API! Use /predict to generate text."}

# ✅ Favicon Route to Prevent 404 Errors
@app.get("/favicon.ico")
async def favicon():
    return FileResponse("favicon.ico")  # You can add a real favicon if needed

# Define request body structure
class InputData(BaseModel):
    prompt: str  # User input prompt for inference

# API Route for inference
@app.post("/predict")
async def generate_text(data: InputData):
    input_text = data.prompt
    inputs = tokenizer(input_text, return_tensors="pt")
    with torch.no_grad():
        output = model.generate(**inputs, max_length=512)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return {"response": generated_text}

# ✅ Start Ngrok tunnel
ngrok_tunnel = ngrok.connect(8000, "http")
print(f"🚀 PUBLIC API URL: {ngrok_tunnel.public_url}")

# Run the FastAPI server
uvicorn.run(app, host="0.0.0.0", port=8000)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🚀 PUBLIC API URL: https://0611-35-204-29-204.ngrok-free.app


INFO:     Started server process [1681]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     54.86.50.139:0 - "GET / HTTP/1.1" 200 OK
